# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import sys
!{sys.executable} -m pip install lxml

In [ ]:
# The library used in Coursera as of Apr-2023 splits differently the cities from wiki

In [119]:
# tuple of team names that should be modified and to prepare for split
# NOT list cause we want to use it with startswith / endswith
tls = (
    # nhl
    'Tampa Bay', 
    'New Jersey',
    'San Jose',
    'Los Angeles',
    'New York',
    'St. Louis',
    'Maple Leafs',
    'Red Wings',
    'Blue Jackets',
    'Golden Knights',
    # nba
    'Golden State',
    'Trail Blazers',
    'Oklahoma City',
    'New Orleans',
    'San Antonio',
    # mlb
    'Blue Jays',
    'White Sox',
    'Red Sox',
    'Kansas City',
    'San Francisco',
    'San Diego',
    # nfl
    'Green Bay',
    'New England'
    
)

# Correct metropolitan area name so that to match the one in csv file
map_cities = {
    'New York City': 'New York',
    'San Francisco Bay Area': 'San Francisco',
    'Dallas–Fort Worth':'Dallas',
    'Washington, D.C.':'Washington',
    'Minneapolis–Saint Paul':'Minneapolis',
    'Miami–Fort Lauderdale':'Miami',
    'Tampa Bay Area':'Tampa Bay',
    'Las Vegas': 'Vegas',
    # for nfl that is not correct geographically wise
#     'Raleigh': 'Charlotte'
} 

map_team_cities = {
    # nhl
    'Minnesota': 'Minneapolis',
    'Arizona': 'Phoenix',
    'San Jose': 'San Francisco',
    'New Jersey': 'New York',
    'Anaheim': 'Los Angeles',
#     'Carolina': 'Raleigh',
    'Carolina': 'Charlotte',
    'Colorado': 'Denver',
    'Florida':'Miami',
    # nba
    'Brooklyn' : 'New York',
    'Golden State': 'San Francisco',
    'Indiana': 'Indianapolis',
    'Utah': 'Salt Lake City',
    # mlb
    'Oakland': 'San Francisco',
    'Texas': 'Dallas',
    # nfl
    'Tennessee': 'Nashville',
    'New England': 'Boston',
#     'Raleigh': 'Charlotte'
}    

In [120]:
# keep columns for csv data
q1 = ['Metropolitan area', 'Team Name', 'W', 'L']
q2 = ['Metropolitan area', 'Team Name', 'W', 'L', 'W/L%']
q3 = ['Metropolitan area', 'Team Name', 'W', 'L', 'W-L%']
q4 = ['Metropolitan area', 'Team Name', 'W', 'L', 'W-L%']

# keep column for wikipedia data df
qa1 = ['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']
qa2 = ['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']
qa3 = ['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']
qa4 = ['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']

In [121]:
# auxiliary function to help with NBA df
def custom_split(l):
    l.pop()
    return ' '.join(l)

In [122]:
def data_cleansing(df, year, major_league):
    
    # work only on 2018 from now on
    df = df[df['year'] == year].dropna()

    # rename columns
    if major_league == 'NBA':
        df = df.rename(columns={'W/L%': 'Win/Loss Ratio'}) 
    elif major_league in ['MLB', 'NFL']:
        df = df.rename(columns={'W-L%': 'Win/Loss Ratio'})

    if major_league == 'NHL':
        # find index of divisions
        # the following returns an Index object NOT a list => be careful
        divisions = df[df['team'].str.contains('Division') & (df['year'] == year)].dropna().index
        # check this out:
        # <class 'pandas.core.indexes.numeric.Int64Index'>
        # print(type(division))

        # loop the indices (divisions.values is a LIST)
        cnt = 0
        for div in divisions.values:
            if cnt != len(divisions.values)-1:
                df.loc[list(range(divisions.values[cnt]+1,divisions.values[cnt+1])), 'Division'] = df[df['year'] == year]['team'][div].split()[0]
                cnt += 1
            else:
                df.loc[list(range(divisions.values[cnt]+1,df.shape[0])), 'Division'] = df[df['year'] == year]['team'][div].split()[0]

        # remove also asterisk
        df['team'] = df['team'].str.strip('*')
    elif major_league == 'NBA':
        df['team'] = df[df['year']==year]['team'].str.split().apply(custom_split).str.strip('*')
    elif major_league == 'NFL':
        
        # find index of AFCs / NFCs => C stands for conference
        # the following returns an Index object NOT a list => be careful
        conferences = df[(df['team'].str.contains('AFC') | df['team'].str.contains('NFC')) & (df['year'] == year)].dropna().index
        # check this out:
        # <class 'pandas.core.indexes.numeric.Int64Index'>

        # loop the indices (divisions.values is a LIST)
        cnt = 0
        for conference in conferences.values:
            if cnt != len(conferences.values)-1:
                df.loc[list(range(conferences.values[cnt]+1,conferences.values[cnt+1])), 'Conference'] = df[df['year'] == year]['team'][conference].split()[0]
                cnt += 1
            else:
                df.loc[list(range(conferences.values[cnt]+1,df.shape[0])), 'Conference'] = df[df['year'] == year]['team'][conference].split()[0]
        
        df['team'] = df['team'].str.strip('*+')
        
        
    # create an auxiliary flag to define the cases for split
    # another way to do it more pythonic?
    df['aux_split_flag'] = np.where(df['team'].str.startswith(tls), 0,
                                        np.where(df['team'].str.endswith(tls), 1, 99)
                                    )

    # find index of rows for which team consists of 2 words (i.e. team and city)
    index_normal = df[df['aux_split_flag'] == 99].index
    # find index of rows for which team consists of 3 words (i.e. team(1) and city(2))
    index_startswith = df[df['aux_split_flag'] == 0].index
    # find index of rows for which team consists of 3 words (i.e. team(2) and city(1))
    index_endswith = df[df['aux_split_flag'] == 1].index

    # split by patterns per case
    pattern_normal_city = r'(^(?:\S+\s+){1})'
    pattern_normal_team = r'(\S\w+$)'
    pattern_startswith = r'(^(?:\S+\s+){2})'
    pattern_endswith = r'(\w+\s\w+$)'

    # for each of the rows with fullfil the index found above split team name and metro area in different columns
    df.loc[index_normal.union(index_endswith),'Metropolitan area'] = df['team'].str.extract(pattern_normal_city).squeeze()
    df.loc[index_startswith,'Metropolitan area'] = df['team'].str.extract(pattern_startswith).squeeze()

    df.loc[index_normal.union(index_startswith), 'Team Name'] = df['team'].str.extract(pattern_normal_team).squeeze()
    df.loc[index_endswith, 'Team Name'] = df['team'].str.extract(pattern_endswith).squeeze()
    
    # strip trainling spaces
    df['Metropolitan area'] = df['Metropolitan area'].str.strip()

    # and change values for two dataframes to match
    df.replace({'Metropolitan area':map_team_cities}, inplace=True)
    
    # return df without the auxiliary column
    return df.drop('aux_split_flag', axis=1)

In [123]:
def set_cities(df, col, major_league):
    
    # keep specific columns for q1 and rename population
    df = df[col].rename(columns={'Population (2016 est.)[8]':'Population'}) 

    # remove [note]

    # regex to find text within brackets from df
    pattern_remove_note = r'\[(.*?)\]'
    df.replace(regex= pattern_remove_note, value = '', inplace=True)

    # split multiple teams within one row to multiple rows with same index
    pattern_split_teams = r'[0-9]+[a-z]+|[A-Z][a-z]*'

    # split values into a list
    # if the team name contains two words, then assign a value 99 cause we do not want to split it
    
    # special case: MLB: CubsWhite Sox
    # we could create a regex to cover all cases; but we do not know now
    # so we kind of doing it manually by utilizing lists
    # pop() removes the last eelem
    if major_league == 'MLB':
        str_to_modify = 'CubsWhite Sox'
        auxl = str_to_modify.split()
        idx = df.index[df[major_league] == str_to_modify].tolist()[0]
        string_to_add = auxl.pop()
        df[major_league].loc[idx] = auxl[0]

    
    df['Team Name'] = df[major_league].str.findall(pattern_split_teams). \
                                            where(df[major_league].apply(lambda n: len(n.split())) == 1).fillna(99)
    
    # for the teams having two words use the original value => which is not a list
    df['Team Name'] = np.where(df['Team Name'] == 99, df[major_league], df['Team Name'])

    # Rows that contain a list will be exploded to multiple rows one per each list value with same index
    df = df.explode('Team Name')
    
    # add new string for MLB
    if major_league == 'MLB':
        # we use the actual position of the string within th series
        # cause there are two elements with the same index
        # is there another way?
        df['Team Name'].iloc[7] = df['Team Name'].loc[idx].tolist()[1]+ ' ' + string_to_add

    # and replace where necessary
    df.replace({'Metropolitan area':map_cities}, inplace=True)

    #finally replace any empty strings to NaN
    df['Team Name'].replace('', np.nan, inplace=True)

    # finally dropna and return
    return df.dropna()

In [124]:
def get_final(df1, df2, year, major_league, columns_teams, columns_cities):
    
    cols_nhl = ['Metropolitan area', 'Team Name', 'W', 'L']
    cols_other = ['Metropolitan area', 'Team Name', 'W', 'L', 'Win/Loss Ratio']
    
    if major_league == 'NHL':
        df_teams = data_cleansing(df1, year, major_league)[cols_nhl]
    else:
        df_teams = data_cleansing(df1, year, major_league)[cols_other]
    
    df_cities = set_cities(df2, columns_cities, major_league)
    
    # join two dataframes
    merged = pd.merge(df_teams, df_cities, on=['Team Name', 'Metropolitan area'])
    
    # calculate win / loss ration for each team
    # and set index to metro area so that datafranes can be joined easier after calculating average w/l ratio
    if major_league == 'NHL':
        # modify the columns 'W' / 'L' / 'Population' to integers to make calc for NHL
        merged = merged.astype({'W': 'int32', 'L': 'int32', 'Population': 'int32'})
        merged['Win/Loss Ratio'] = merged['W'] / (merged['W'] + merged['L'])
    else:
        merged = merged.astype({'W': 'int32', 'L': 'int32', 'Population': 'int32', 'Win/Loss Ratio': 'float'})

    merged.set_index('Metropolitan area', inplace=True)
    
    # for teams under the same metro area replace w/l ration with average
    df_win_loss_mean = merged.groupby('Metropolitan area').agg({"Win/Loss Ratio":np.nanmean}).reset_index()

    # merge mean with initial dataframe
    df_final = pd.merge(merged, df_win_loss_mean, on='Metropolitan area').\
                                rename(columns={'Win/Loss Ratio_y': 'Win/Loss Ratio',
                                                # this is neede for validation purposes and after all OK
                                                # is removed
#                                                 'Win/Loss Ratio_x': 'Team Win/Loss Ratio',
                                                'W': 'Wins',
                                                'L': 'Losses'
                                               }
                                      )
    
    # we are keeping the columns that we need for the pearson correlation
    # Population and average w/l ration per metro area
    return df_final.drop(['Win/Loss Ratio_x', 'Team Name', 'Wins', 'Losses'], axis=1).drop_duplicates()

In [125]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    final_nhl = get_final(nhl_df,
                      cities,
#                       data_cleansing(nhl_df, 2018, 'NHL')[q1], 
#                       set_cities(cities, qa1, 'NHL'), 
                      2018, 
                      'NHL', 
                      q1,
                      qa1
                     )

#     raise NotImplementedError()
    population_by_region = final_nhl['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = final_nhl['Win/Loss Ratio'].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    p_coeff, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return p_coeff

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [126]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
   
def nba_correlation():
    # YOUR CODE HERE
    final_nba = get_final(nba_df, 
                      cities, 
                      2018, 
                      'NBA', 
                      q2, 
                      qa2
                     )
#     raise NotImplementedError()

    population_by_region = final_nba['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = final_nba['Win/Loss Ratio'].tolist() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    p_coeff, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return p_coeff

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [127]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    final_mlb = get_final(mlb_df, 
                          cities, 
                          2018, 
                          'MLB', 
                          q3, 
                          qa3
                         )
#     raise NotImplementedError()
    
    population_by_region = final_mlb['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = final_mlb['Win/Loss Ratio'].tolist() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    p_coeff, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return p_coeff

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [142]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    final_nfl = get_final(nfl_df, 
                      cities, 
                      2018, 
                      'NFL', 
                      q4, 
                      qa4
                     )
#     raise NotImplementedError()
    
    population_by_region = final_nfl['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = final_nfl['Win/Loss Ratio'].tolist() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    p_coeff, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return p_coeff

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [159]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
# from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
# Function which returns subset or r length from n
from itertools import combinations
from itertools import combinations_with_replacement
from itertools import permutations
from itertools import starmap

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def calc_p_value(df1, df2):
    
    p_df = pd.merge(df1, df2, on='Metropolitan area')

    # run ttest
#     t_stat, p_value = ttest_ind(p_df['Win/Loss Ratio_x'], p_df['Win/Loss Ratio_y'])
    t_stat, p_value = ttest_rel(p_df['Win/Loss Ratio_x'], p_df['Win/Loss Ratio_y'])
    return p_value

def sports_team_performance():
    # YOUR CODE HERE
    # re - create the data frames of the previous questions
    final_nhl = get_final(nhl_df, 
                      cities, 
                      2018, 
                      'NHL', 
                      q1, 
                      qa1
                     )
    final_nba = get_final(nba_df, 
                      cities, 
                      2018, 
                      'NBA', 
                      q2, 
                      qa2
                     )
    final_mlb = get_final(mlb_df, 
                      cities, 
                      2018, 
                      'MLB', 
                      q3, 
                      qa3
                     )

    final_nfl = get_final(nfl_df, 
                      cities, 
                      2018, 
                      'NFL', 
                      q4, 
                      qa4
                     )

    # create list these so that to calculate the permutations of each league pair
    # the order IS IMPORTANT so that to create the final dataframe as per the instructions
    ls = [final_nfl, final_nba
      , final_nhl, final_mlb
     ]

#     raise NotImplementedError()
    
    # permutations creates pairs of the ls list (the list of data frames)
    # for example (final_nfl, final_nba), (final_nfl, final_nhl), etc.
    # it returns an object, which then we transform it to a list
    li = list(permutations(ls, 2))
    
    # for each pair calculate the p_value
    # starmap calls the custom function for every pair of dataframes
    # the first element of the tuple (e.g. final_nfl) is used as the first parameter 
    # of the custom function and the second element as the second parameter
    league_permutations = list(starmap(calc_p_value, li))
     
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    # in order to assign values in each cell of the dataframe the logic is the following:
    # 
    i = 0
    for row in p_values.itertuples():
        cnt = i
        for col in p_values.columns:
            if row.Index == col:
                continue
            else:
                p_values.at[row.Index, col] = league_permutations[cnt]
                cnt = cnt + 1
        i = i + 3
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    return p_values
#     return calc_p_value(final_nba, final_nhl)